In [ ]:
# Configuração inicial do ambiente Kedro
import os
import sys
from datetime import datetime
import pandas as pd
import numpy as np

# Verificar se estamos no ambiente Kedro
print(f"Diretório atual: {os.getcwd()}")
print(f"Ambiente Kedro - {datetime.now()}")
print(f"Python version: {sys.version}")
print(f"Pandas version: {pd.__version__}")

# Verificar se o Kedro está disponível
try:
    import kedro

    print(f"✅ Kedro version: {kedro.__version__}")
except ImportError:
    print("❌ Kedro não encontrado. Execute: pip install kedro")

In [ ]:
# Carregar extensão Kedro para comandos mágicos
%load_ext kedro.extras.extensions.ipython

# Verificar se a extensão foi carregada
print("Extensão Kedro carregada com sucesso!")
print("Comandos mágicos disponíveis:")
print("- %reload_kedro")
print("- %kedro_catalog")
print("- %kedro_lineage")
print("- %kedro_replay")


In [ ]:
# Mostrar informações do projeto
%reload_kedro

# Verificar pipelines disponíveis
print("\nPipelines disponíveis:")
print("- data_engineering")
print("- __default__")


In [ ]:
# Listar todos os datasets disponíveis
%kedro_catalog

# Mostrar datasets de um pipeline específico
%kedro_catalog --pipeline data_engineering


In [ ]:
# Acessar dados usando o contexto Kedro
from kedro.framework.startup import bootstrap_project
from kedro.framework.context import KedroContext

# Inicializar contexto do projeto
bootstrap_project(os.getcwd())
context = KedroContext(os.getcwd())

# Listar datasets disponíveis
print("Datasets disponíveis:")
for dataset_name in context.catalog.list():
    print(f"- {dataset_name}")

# Carregar um dataset específico
try:
    measurements_data = context.catalog.load("measurements_data")
    print(f"\nDataset 'measurements_data' carregado:")
    print(f"Tipo: {type(measurements_data)}")
    if hasattr(measurements_data, "shape"):
        print(f"Shape: {measurements_data.shape}")
    elif hasattr(measurements_data, "__len__"):
        print(f"Tamanho: {len(measurements_data)}")
except Exception as e:
    print(f"Erro ao carregar dataset: {e}")

In [ ]:
# Acessar dados usando o contexto Kedro
from kedro.framework.startup import bootstrap_project
from kedro.framework.context import KedroContext

# Inicializar contexto do projeto
bootstrap_project(os.getcwd())
context = KedroContext(os.getcwd())

# Listar datasets disponíveis
print("Datasets disponíveis:")
for dataset_name in context.catalog.list():
    print(f"- {dataset_name}")

# Carregar um dataset específico
try:
    measurements_data = context.catalog.load("measurements_data")
    print(f"\nDataset 'measurements_data' carregado:")
    print(f"Tipo: {type(measurements_data)}")
    if hasattr(measurements_data, "shape"):
        print(f"Shape: {measurements_data.shape}")
    elif hasattr(measurements_data, "__len__"):
        print(f"Tamanho: {len(measurements_data)}")
except Exception as e:
    print(f"Erro ao carregar dataset: {e}")

In [ ]:
# Executar pipeline completo usando comando mágico
%kedro_replay --pipeline data_engineering

# Ou executar via contexto
print("Executando pipeline data_engineering...")
try:
    context.run(pipeline_name="data_engineering")
    print("✅ Pipeline executado com sucesso!")
except Exception as e:
    print(f"❌ Erro na execução: {e}")


In [ ]:
# Executar pipeline com parâmetros específicos
from kedro.framework.context import KedroContext

# Recarregar contexto
context = KedroContext(os.getcwd())

# Executar com parâmetros
params = {"start_date": "2025-01-01", "end_date": "2025-08-07", "batch_size": 1000}

print("Executando pipeline com parâmetros personalizados...")
try:
    context.run(
        pipeline_name="data_engineering",
        tags=["data_ingestion"],
        node_names=["fetch_measurements", "process_data"],
    )
    print("✅ Pipeline executado com sucesso!")
except Exception as e:
    print(f"❌ Erro na execução: {e}")

In [ ]:
# Visualizar linhagem de dados
%kedro_lineage measurements_data

# Mostrar dependências entre datasets
print("\nDependências dos datasets:")
for dataset_name in ["measurements_data", "measurements_dataframe", "catalog_save_result"]:
    try:
        %kedro_lineage {dataset_name}
    except Exception as e:
        print(f"Erro ao mostrar linhagem de {dataset_name}: {e}")


In [ ]:
# Executar apenas nodes específicos
print("Executando nodes específicos...")

try:
    # Executar apenas o node de autenticação
    context.run(pipeline_name="data_engineering", node_names=["authenticate_scgas"])
    print("✅ Node de autenticação executado!")

    # Executar apenas o node de coleta de dados
    context.run(pipeline_name="data_engineering", node_names=["fetch_measurements"])
    print("✅ Node de coleta executado!")

except Exception as e:
    print(f"❌ Erro na execução: {e}")

In [ ]:
# Verificar logs de execução
import logging

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Executar com logging detalhado
print("Executando pipeline com logging detalhado...")

try:
    context.run(
        pipeline_name="data_engineering",
        tags=["data_ingestion"],
        node_names=["fetch_measurements", "process_data", "save_results"],
    )
    print("✅ Pipeline executado com sucesso!")

    # Verificar datasets resultantes
    print("\nDatasets após execução:")
    for dataset_name in [
        "measurements_data",
        "measurements_dataframe",
        "catalog_save_result",
    ]:
        try:
            data = context.catalog.load(dataset_name)
            print(f"- {dataset_name}: {type(data).__name__}")
        except Exception as e:
            print(f"- {dataset_name}: Erro ao carregar - {e}")

except Exception as e:
    print(f"❌ Erro na execução: {e}")

In [ ]:
# Execução condicional baseada em parâmetros
from kedro.framework.context import KedroContext

context = KedroContext(os.getcwd())

# Verificar se dados já existem
try:
    existing_data = context.catalog.load("measurements_data")
    data_exists = len(existing_data) > 0 if hasattr(existing_data, "__len__") else True
    print(f"Dados existentes: {data_exists}")

    if not data_exists:
        print("Executando pipeline para coletar dados...")
        context.run(pipeline_name="data_engineering")
    else:
        print("Dados já existem. Pulando execução do pipeline.")

except Exception as e:
    print(f"Executando pipeline (erro ao verificar dados existentes): {e}")
    context.run(pipeline_name="data_engineering")

In [ ]:
# Limpeza de recursos
print("Finalizando execução...")

# Verificar status final
print("\nStatus final dos datasets:")
for dataset_name in [
    "measurements_data",
    "measurements_dataframe",
    "catalog_save_result",
]:
    try:
        data = context.catalog.load(dataset_name)
        if hasattr(data, "shape"):
            print(f"- {dataset_name}: {data.shape}")
        elif hasattr(data, "__len__"):
            print(f"- {dataset_name}: {len(data)} itens")
        else:
            print(f"- {dataset_name}: {type(data).__name__}")
    except Exception as e:
        print(f"- {dataset_name}: Não disponível - {e}")

print("\n✅ Notebook de comandos mágicos Kedro concluído!")